In [ ]:
data = """ Jack and Jill went up the hill\n
To fetch a pail of water\n
Jack fell down and broke his crown\n
And Jill came tumbling after\n """

In [ ]:
data = """চাষের জমিতে পুঁজিবাদ আবাদ করে কৃষক,না বুঝেই।
তার শরীর বেয়ে গড়িয়ে পড়া লবণাক্ত ঘাম থেকে লবণ আলাদা করে চড়া মূল্যে আবার বিক্রি করা হয়, তার কাছেই।
অভাগা  কৃষক, না বোঝে গণতন্ত্র, না সমাজতন্ত্র। শুধু চেয়ে ছিল একদিন একটু আবাদ করবে স্বচ্ছলতা।
. \n """

In [ ]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
from keras_preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


# one word in , one word out

In [ ]:
t = Tokenizer()
t.fit_on_texts([data])
encoded = t.texts_to_sequences([data])[0]
vocab_size = len(t.word_index) + 1

In [ ]:
seq = []

for i in range(1,len(encoded)):
  s = encoded[i-1:i+1 ]
  seq.append(s)

In [ ]:
sequence = np.array(seq)

In [ ]:
X = sequence[:,0]
y = sequence[:,1]
y = to_categorical(y, num_classes = vocab_size)

In [ ]:
def build_model(vocab_size):
  model = Sequential()
  model.add(Embedding(vocab_size,10,input_length = 1))
  model.add(LSTM(75))
  model.add(Dense(vocab_size , activation = 'softmax'))

  model.compile(loss = 'categorical_crossentropy',optimizer = 'adam' , metrics = ['accuracy'])
  model.summary()
  plot_model(model, to_file='model.png', show_shapes=True)

  return model

In [ ]:
model = build_model(vocab_size)
model.fit(X,y,epochs=500)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 1, 10)             220       
                                                                 
 lstm_12 (LSTM)              (None, 75)                25800     
                                                                 
 dense_8 (Dense)             (None, 22)                1672      
                                                                 
Total params: 27,692
Trainable params: 27,692
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 3.0910 - accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 14ms/step - loss: 3.0901 - accuracy: 0.0417
Epoch 3/500
1/1 [==============================] - 0s 11ms/step - loss: 3.0892 - accuracy: 0.0833
Epoch 4/500
1/1 [

In [ ]:
def gen_seq(model,t,seed,n_words):
  it , r = seed, seed

  for _ in range(n_words):
    encode = np.array(t.texts_to_sequences([it])[0])
    yhat = np.argmax(model.predict(encode))



    out = ''

    for word,index in t.word_index.items():

      if index == yhat:
        out = word
        break

    it = out

    r = r + ' ' + out





  return r

In [ ]:
seed = 'fell'
out = gen_seq(model,t,seed,8
              )
print(out)

1/1 [==============================] - 0s 20ms/step
fell down and jill went up the hill to


# Line by Line Sequence

In [ ]:
sq = []

for line in data.split('\n'):
  encoded = t.texts_to_sequences([line])[0]

  for i in range(1,len(encoded)):

    sq.append(encoded[:i+1])







In [ ]:
from keras_preprocessing.sequence import pad_sequences
max_len = max(len(s) for s in sq)
pad_sq = pad_sequences(sq,maxlen = max_len , padding = 'pre')

In [ ]:
X,y = pad_sq[:,:-1] , pad_sq[:,-1:]
y = to_categorical(y,num_classes = vocab_size)

In [ ]:
def build_model(vocab_size,max_len):

  model = Sequential()
  model.add(Embedding(vocab_size,10,input_length = max_len - 1))
  model.add(LSTM(50))
  model.add(Dense(vocab_size,activation = 'softmax'))

  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])
  model.summary()

  return model

In [ ]:
model = build_model(vocab_size,max_len)
model.fit(X,y,epochs = 350)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 6, 10)             220       
                                                                 
 lstm_5 (LSTM)               (None, 50)                12200     
                                                                 
 dense_5 (Dense)             (None, 22)                1122      
                                                                 
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
Epoch 1/350
1/1 [==============================] - 3s 3s/step - loss: 3.0926 - accuracy: 0.0000e+00
Epoch 2/350
1/1 [==============================] - 0s 11ms/step - loss: 3.0909 - accuracy: 0.0000e+00
Epoch 3/350
1/1 [==============================] - 0s 13ms/step - loss: 3.0892 - accuracy: 0.0476
Epoch 4/350
1/

In [ ]:
def gen_seq(model,t,m,seed,n_words):
  it  = seed

  for _ in range(n_words):
    encode = np.array(t.texts_to_sequences([it])[0])
    encode = pad_sequences([encode],maxlen = m-1, padding = 'pre')
    yhat = np.argmax(model.predict(encode))



    out = ''

    for word,index in t.word_index.items():

      if index == yhat:
        out = word
        break

    #it = out

    it +=  ' ' + out





  return it

In [ ]:
gen_seq(model,t,max_len-1,'Jack',6)

1/1 [==============================] - 0s 29ms/step


'Jack and went up the hill hill'

# three_words in , one word out


In [ ]:
seq = []
t  = tokenizer

encoded = t.texts_to_sequences([data])[0]

for i in range(3,len(encoded)):

  seq.append(encoded[i-3:i+1])

In [ ]:
from keras_preprocessing.sequence import pad_sequences
max_len = max(len(s) for s in seq)
pad_seq = pad_sequences(seq,maxlen = max_len , padding = 'pre')

In [ ]:
X,y = pad_seq[:,:-1] , pad_seq[:,-1:]
y = to_categorical(y,num_classes = vocab_size)

In [ ]:
def build_model(vocab_size,max_len):

  model = Sequential()
  model.add(Embedding(vocab_size,10,input_length = max_len - 1))
  model.add(LSTM(50))
  model.add(Dense(vocab_size,activation = 'softmax'))

  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])
  model.summary()

  return model

In [ ]:
def gen_seq(model,t,m,seed,n_words):
  it  = seed

  for _ in range(n_words):
    encode = np.array(t.texts_to_sequences([it])[0])
    encode = pad_sequences([encode],maxlen = m-1, padding = 'pre')
    yhat = np.argmax(model.predict(encode))



    out = ''

    for word,index in t.word_index.items():

      if index == yhat:
        out = word
        break

    #it = out

    it +=  ' ' + out





  return it

In [ ]:
model = build_model(vocab_size , max_len)
model.fit(X,y,epochs = 360)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 3, 10)             5030      
                                                                 
 lstm_8 (LSTM)               (None, 50)                12200     
                                                                 
 dense_8 (Dense)             (None, 503)               25653     
                                                                 
Total params: 42,883
Trainable params: 42,883
Non-trainable params: 0
_________________________________________________________________
Epoch 1/360
22/22 [==============================] - 5s 89ms/step - loss: 6.2227 - accuracy: 0.0030
Epoch 2/360
22/22 [==============================] - 1s 38ms/step - loss: 6.2165 - accuracy: 0.0222
Epoch 3/360
22/22 [==============================] - 0s 22ms/step - loss: 6.2118 - accuracy: 0.0236
Epoch 4/360
22

In [ ]:
gen_seq(model,t,max_len,'বর্ষা আকাশ বিষণ্ণ মনে', 20)

1/1 [==============================] - 0s 19ms/step


'বর্ষা আকাশ বিষণ্ণ মনে এই শালিকের ভোরে দিনেই। ভালোবাসার জমা উপেক্ষা লিখে আর শালিক। তুমি বাড়লে অপেক্ষায় মতো ডাহুকের বদলায়নি নিয়ম শোনো এপিটাফের কোলে'

# NLM for text generation


In [ ]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
import string
import re
from keras_preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import numpy as np
from keras_preprocessing.sequence import pad_sequences

In [ ]:
#functions that we are gonna use in our later code

def load_doc(filename):
  file = open(filename,'r')

  text = file.read()
  file.close()

  return text

def clean_doc(doc):
# replace '--' with a space ' '
  doc = doc.replace('--', ' ')
# split into tokens by white space
  tokens = doc.split()
# prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
  tokens = [re_punc.sub('', w) for w in tokens]
# remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
# make lower case
  #tokens = [word.lower() for word in tokens]
  return tokens

def save_doc(lines, filename):
  data = '\n'.join(lines)
  file = open(filename, 'w')
  file.write(data)
  file.close()


In [ ]:
data = load_doc('/content/New Text Document.txt')

#clean_data = clean_doc(data)

clean_data = data.split()

In [ ]:
#clean_data

In [ ]:
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
tokens = [re_punc.sub('', w) for w in clean_data]

In [ ]:
def make_sequence(clean_data , length):

  seq = []

  for i in range(length,len(clean_data)):

    s = clean_data[i-length:i]
    line = ' '.join(s)

    seq.append(line)

  return seq


In [ ]:
seq = make_sequence(tokens,length = 6)
save_doc(seq, 'seq_data')

In [ ]:
def pad_or_truncate(lst, length=6, padding_value=0):
    if len(lst) < length:
        return lst + [padding_value] * (length - len(lst))
    elif len(lst) > length:
        return lst[:length]
    else:
        return lst

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(seq)
nw_sq = tokenizer.texts_to_sequences(seq)

In [ ]:
nw_sq_fixed_length = [pad_or_truncate(seq, length=6) for seq in nw_sq]

In [ ]:
encoded_sequences = np.array(nw_sq_fixed_length)


In [ ]:
encoded_sequences

array([[495, 493, 492,  83,   1,  43],
       [493, 492,  83,   1,  43,   2],
       [492,  83,   1,  43,   2,  84],
       ...,
       [487, 488, 489, 490, 491,   2],
       [488, 489, 490, 491,   2, 494],
       [489, 490, 491,   2, 494, 496]])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
vocab_size

503

In [ ]:
X,y = encoded_sequences[:,:-1],encoded_sequences[:,-1:]
y = to_categorical(y,num_classes = vocab_size)
seq_len = X.shape[1]


In [ ]:
def build_model(vocab_size,seq_len):
  model = Sequential()
  model.add(Embedding(vocab_size,50,input_length = seq_len))
  model.add(LSTM(100,return_sequences=True))
  model.add(LSTM(100))
  model.add(Dense(100,activation = 'relu'))
  model.add(Dense(vocab_size,activation = 'softmax'))

  model.compile(loss = 'categorical_crossentropy',optimizer = 'adam' , metrics = ['accuracy'])


  model.summary()

  return model

In [ ]:
model1 = build_model(vocab_size,seq_len)
model1.fit(X,y,epochs = 350)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 5, 50)             24850     
                                                                 
 lstm_10 (LSTM)              (None, 5, 100)            60400     
                                                                 
 lstm_11 (LSTM)              (None, 100)               80400     
                                                                 
 dense_10 (Dense)            (None, 100)               10100     
                                                                 
 dense_11 (Dense)            (None, 497)               50197     
                                                                 
Total params: 225,947
Trainable params: 225,947
Non-trainable params: 0
_________________________________________________________________
Epoch 1/350
22/22 [=============================

In [ ]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
  result = []
  in_text = seed_text
# generate a fixed number of words
  for _ in range(n_words):
# encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    #encoded = pad_or_truncate(encoded, length=6)
# truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

# predict probabilities for each word
    yhat = np.argmax(model.predict(encoded))
# map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
# append to input
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)


In [ ]:
txt = generate_seq(model, tokenizer, seq_len, 'রক্তবমির', 100)
print(txt)


1/1 [==============================] - 0s 19ms/step
ছুঁয়ে ছুঁয়ে ছুঁয়ে ছুঁয়ে এক আলো আলো আলো আলো এক আবাদ আবাদ আবাদ এক এখন চেয়ে তার রেখে রেখে দ্রোণাচার্যের তোমাকে করা মতো অথবা আর্যপ্রেমিক। চেখে রোজ এই কবির আলো রোজ চেনা ভিতর প্রেম তোমার জমা কবিতার বোধ। নিলামে তো যাওয়া আমার তুমি না নাগরিক দিনেই। লিখে শ্রমিকের মনে পারলে শূন্যতার দল স্বচ্ছলতা। অথবা ঝরে বারবার তেলওয়াত ভুলে কবির থেমে উড়ে রাতের করে শিউলির যাওয়া ভাঙে জীবন ইটটাও চঞ্চল শিউলির যত্ন গেলে গেলে এবং চঞ্চল এখন জমা উচ্চকণ্ঠে মনে এবং থেকে কোলাহলমুখর নামাঙ্কিত কেন সম্ভ্রম। না হচ্ছে পৃথিবী সম্ভ্রম। সংসারে ম্লান দল। ব্যস্ত ব্যস্ত শুধু সংসারের করে আর আর একদিন


In [ ]:
txt

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             '

In [ ]:
from random import randint

seed_text = data[randint(0,len(data))]


In [ ]:
seed_text

' '

# Image captioning


In [ ]:
#!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
from keras.applications.vgg16 import VGG16
# load the model
model = VGG16()


553467096/553467096 [==============================] - 17s 0us/step


In [ ]:
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions



In [ ]:
img = load_img('/content/cfe.jpg',target_size=(224, 224))
img = img_to_array(img)

img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
img = preprocess_input(img)

In [ ]:
yhat = model.predict(img)


1/1 [==============================] - 8s 8s/step


In [ ]:
from keras.applications.vgg16 import decode_predictions
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))


35363/35363 [==============================] - 0s 0us/step
coffee_mug (68.00%)


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']]
candidate = ['the', 'fast', 'brown', 'jackle', 'jumped', 'over', 'the']
score = sentence_bleu(reference, candidate)
print(score)


4.2874310268234296e-78


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


# Machine Translation(NMT-encoder-decoder)

In [ ]:
import string
import re
from pickle import dump
from unicodedata import normalize
import numpy as np

In [ ]:
def load_doc(filename):

  file = open(filename,mode = 'rt' , encoding ='utf-8')

  text = file.read()

  file.close()

  return text

def to_pairs(doc):
  lines = doc.strip().split('\n')
  pairs = [line.split('\t') for line in lines]
  return pairs

def clean_pairs(lines):
  cleaned = list()
# prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  re_print = re.compile('[^%s]' % re.escape(string.printable))
  for pair in lines:
    clean_pair = list()
    for line in pair:
# normalize unicode characters
      line = normalize('NFD', line).encode('ascii', 'ignore')

      line = line.decode('UTF-8')
# tokenize on white space
      line = line.split()
# convert to lowercase
      line = [word.lower() for word in line]
# remove punctuation from each token
      line = [re_punc.sub('', w) for w in line]
# remove non-printable chars form each token
      line = [re_print.sub('', w) for w in line]
# remove tokens with numbers in them
      line = [word for word in line if word.isalpha()]
# store as string
      clean_pair.append(' '.join(line))
    cleaned.append(clean_pair)
  return np.array(cleaned)


In [ ]:
filename = '/content/deu.txt'

data = load_doc(filename)


pairs = to_pairs(data)


In [ ]:
clean_data = clean_pairs(pairs)

In [ ]:
clean_data[0][2]

'ccby france attribution tatoebaorg cm roujin'

In [ ]:
nw_clean_data = np.delete(clean_data,2,axis = 1)

In [ ]:
nw_clean_data.shape

(267186, 2)

In [ ]:
n_sentences = 10000
dataset =nw_clean_data[:n_sentences, :]
# random shuffle
np.random.shuffle(dataset)
# split into train/test
train, test = dataset[:9000], dataset[9000:]


In [ ]:
test.shape

(1000, 2)